# Training

Experiments were conducted using:
- `mb_vae_dti/training/run.py`: command line interface for training
- and the scripts in `scripts/training/` for running the experiments

This notebook shows some plots and analysis of the unsupervised pretraining, general DTI training and benchmark fine-tuning results.

In [57]:
from resolve import *

In [158]:
from mb_vae_dti.validating.analysis import *

df = load_gridsearch_results("notebooks/results/summer/version3/multi_modal_kiba_rand")

if "metadata.data.provenance_cols" in df.columns:
    df["metadata.data.provenance_cols"] = df["metadata.data.provenance_cols"].apply(lambda x: x[0])
if "metadata.data.drug_features" in df.columns:
    df = df.drop(columns=["metadata.data.drug_features", "metadata.data.target_features"])

df

2025-07-27 19:35:42,146 - mb_vae_dti.validating.analysis - INFO - Successfully loaded 515 results with 40 columns


,experiment_name,best_val_loss,best_epoch,trainable_params,timestamp,metadata.training.max_epochs,metadata.training.early_stopping_patience,metadata.training.scheduler,metadata.training.learning_rate,metadata.data.load_in_memory,...,metrics.val/ci,metrics.val/mse,metrics.val/pearson,metrics.val/r2,metrics.val/rmse,timing.total_training_time,timing.avg_time_per_epoch,timing.total_epochs,timing.min_epoch_time,timing.max_epoch_time
0,multimodal_KIBA_rand_b10c0000_1753169636,0.207219,95,4805952,1.753176e+09,100,10,one_cycle,0.0010,True,...,0.843356,0.208251,0.832554,0.688882,0.549775,6705.806756,67.058068,100,65.561920,69.350585
1,multimodal_KIBA_rand_b09c0033_1753295642,0.232787,95,10402752,1.753308e+09,100,10,one_cycle,0.0010,True,...,0.832001,0.233342,0.809183,0.651397,0.566606,12740.058989,127.400590,100,124.531620,130.346525
2,multimodal_KIBA_rand_b06c0007_1753213849,0.167601,95,2406400,1.753217e+09,100,10,one_cycle,0.0005,True,...,0.862213,0.167831,0.866917,0.749200,0.498292,3286.418763,32.864188,100,31.528106,33.542326
3,multimodal_KIBA_rand_b10c0004_1753189579,0.208373,91,7210114,1.753200e+09,100,10,one_cycle,0.0005,True,...,0.839699,0.209254,0.831286,0.687384,0.573112,10079.979851,100.799799,100,99.194155,103.582909
4,multimodal_KIBA_rand_b01c0027_1753234754,0.204185,93,10140098,1.753238e+09,100,10,one_cycle,0.0005,True,...,0.841243,0.204798,0.835220,0.694048,0.574030,3264.109634,32.641096,100,30.740782,34.361542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,multimodal_KIBA_rand_b12c0005_1753189320,0.167701,97,7137282,1.753196e+09,100,10,one_cycle,0.0020,True,...,0.860061,0.167893,0.866886,0.749175,0.505087,7036.367263,70.363673,100,66.817396,72.912933
511,multimodal_KIBA_rand_b04c0019_1753214875,0.185055,96,7341698,1.753217e+09,100,10,one_cycle,0.0020,True,...,0.854548,0.185395,0.852237,0.723034,0.544948,2553.782400,25.537824,100,23.509943,26.553761
512,multimodal_KIBA_rand_b06c0034_1753321700,0.190638,97,1789600,1.753324e+09,100,10,one_cycle,0.0020,True,...,0.849416,0.190776,0.846751,0.714996,0.539661,1843.466622,18.434666,100,16.538494,18.906606
513,multimodal_KIBA_rand_b05c0002_1753151600,0.221466,96,2833890,1.753155e+09,100,10,one_cycle,0.0010,True,...,0.829844,0.222029,0.819234,0.668305,0.581632,3276.117473,32.761175,100,30.444456,34.216632


In [159]:
varying_attrs = get_varying_attributes(df)
varying_attrs.remove('metadata.logging.experiment_name')
varying_attrs.extend(["best_val_loss", 'best_epoch', 'trainable_params', "timing.total_training_time"])
varying_attrs

2025-07-27 19:35:42,354 - mb_vae_dti.validating.analysis - INFO - Found 8 varying attributes: ['metadata.training.learning_rate', 'metadata.data.batch_size', 'metadata.logging.experiment_name', 'metadata.model.encoder_kwargs.hidden_dim', 'metadata.model.encoder_kwargs.n_layers', 'metadata.model.embedding_dim', 'metadata.model.encoder_type', 'metadata.model.aggregator_type']


['metadata.training.learning_rate',
 'metadata.data.batch_size',
 'metadata.model.encoder_kwargs.hidden_dim',
 'metadata.model.encoder_kwargs.n_layers',
 'metadata.model.embedding_dim',
 'metadata.model.encoder_type',
 'metadata.model.aggregator_type',
 'best_val_loss',
 'best_epoch',
 'trainable_params',
 'timing.total_training_time']

In [160]:
subdf = get_top_performers(df, n_top=5)[varying_attrs]
# for each columns, rename the column by splitting the column name by "." and taking the last element
subdf.columns = subdf.columns.str.split(".").str[-1]
print(f"Total training time: {(subdf['total_training_time'].sum() / 3600):.2f} hours")
print(f"Average trainable params: {subdf['trainable_params'].mean() / 1000000:.1f}M")
subdf

Total training time: 7.57 hours
Average trainable params: 10.1M


,learning_rate,batch_size,hidden_dim,n_layers,embedding_dim,encoder_type,aggregator_type,best_val_loss,best_epoch,trainable_params,total_training_time
290,0.0005,16,256,3,1024,resnet,concat,0.154958,85,10555136,7962.247344
422,0.0010,32,512,1,1024,resnet,concat,0.155305,83,16484864,2606.612936
300,0.0010,16,256,3,1024,resnet,concat,0.156679,93,10555136,8547.191034
379,0.0020,16,256,1,1024,resnet,concat,0.157262,95,5294336,4868.378374
229,0.0005,32,256,2,768,resnet,concat,0.157600,90,7596544,3258.366287


In [176]:
a = 0.65317
b = 0.62066
c = 0.64136
d = 0.6565
e = 0.64104


total = a + b + c + d + e
print(total / 5)

0.642546


In [183]:
ensemble = """
Name
5 visualized
test/Y_KIBA_ci
test/Y_KIBA_mse
test/Y_KIBA_pearson
test/Y_KIBA_r2
test/Y_KIBA_rmse
•
baseline_finetune_KIBA_rand_ensemble
0.85821
0.16222
0.87049
0.75739
0.40277
•
baseline_finetune_KIBA_rand_ensemble
0.85696
0.16064
0.87211
0.75978
0.4008
•
baseline_finetune_KIBA_rand_ensemble
0.8599
0.15906
0.87356
0.76214
0.39883
•
baseline_finetune_KIBA_rand_ensemble
0.86007
0.15653
0.87542
0.7659
0.39564
•
baseline_finetune_KIBA_rand_ensemble
0.86047
0.15909
0.87372
0.76207
0.39887
"""
from statistics import mean
lines = [line.strip() for line in ensemble.strip().split('\n') if line.strip()]
first_bullet = next(i for i, line in enumerate(lines) if line == '•')
start_idx = 2
column_names = lines[start_idx:first_bullet]
data = {col: [] for col in column_names}

i = first_bullet + 1
while i < len(lines):
    line = lines[i]
    if line == '•' or line.startswith('baseline_'):
        i += 1
        continue
    for j, col_name in enumerate(column_names):
        if i + j < len(lines):
            try:
                value = float(lines[i + j])
                data[col_name].append(value)
            except ValueError:
                break
    
    i += len(column_names)

results = {}
for col_name, values in data.items():
    results[col_name] = mean(values)

results

{'test/Y_KIBA_ci': 0.859122,
 'test/Y_KIBA_mse': 0.159508,
 'test/Y_KIBA_pearson': 0.8730600000000001,
 'test/Y_KIBA_r2': 0.761456,
 'test/Y_KIBA_rmse': 0.399382}

In [184]:
data

{'test/Y_KIBA_ci': [0.85821, 0.85696, 0.8599, 0.86007, 0.86047],
 'test/Y_KIBA_mse': [0.16222, 0.16064, 0.15906, 0.15653, 0.15909],
 'test/Y_KIBA_pearson': [0.87049, 0.87211, 0.87356, 0.87542, 0.87372],
 'test/Y_KIBA_r2': [0.75739, 0.75978, 0.76214, 0.7659, 0.76207],
 'test/Y_KIBA_rmse': [0.40277, 0.4008, 0.39883, 0.39564, 0.39887]}